# UFO Notebook
This Notebook takes the csv version of the UFO database in the file in elections.csv, and turns it into the six tables required for the UFO dashboard in the example.  The dashboard produced is exactly the same as the one in the UFO example; the difference is that Pandas is used to read the csv file and form the tables.  To no one's surprise, this is much shorter and more concise than the equivalent notebook in the ufos directory

In [1]:
import pandas as pd
from galyleo.galyleo_table import GalyleoTable
from galyleo.galyleo_jupyterlab_client import GalyleoClient

Get the UFO sightings for the world and for the US (which has, for whatever reason, almost all the sightings).

In [2]:
ufos = pd.read_csv('ufos.csv')
us = ufos[ufos['country']=='us']

A convenience method that groups the columns in the dataframe by the specified indices, pulls their count, creates a GalyleoTable of the result, and loads the dataframe into the table.

In [3]:
def create_aggregate_dataframe(dataframe, columns, count_column_name = 'count'):
    return dataframe.groupby(columns).size().reset_index(name = count_column_name)

Another convenience method, that takes a dataframe, a table_name, and a client, and wraps the dataframe in a GalyleoTable and sends it to the dashboard via client

In [4]:
def send_dataframe_to_dashboard(dataframe, table_name, client):
    table = GalyleoTable(table_name)
    table.load_from_dataframe(dataframe)
    client.send_data_to_dashboard(table)

The base maps (for the world and the us) far formed by aggregating with respect to year and country, and year and state, respectively

In [5]:
world_map = create_aggregate_dataframe(ufos, ['year', 'country'])
us_map = create_aggregate_dataframe(us, ['year', 'state'])

Create the client and send the data to the dashboard

In [6]:
client = GalyleoClient()
send_dataframe_to_dashboard(world_map, 'aggregate_cy', client)
send_dataframe_to_dashboard(us_map, 'aggregate_sy', client)

The column charts are sightings vs month, with the year selected by the slider and the locale by clicking on the country/state on the map.  Here, we send the groups and they'll be filtered on the dashboard.

In [8]:
world_by_month_and_year = create_aggregate_dataframe(ufos, ['year', 'country', 'month'])
us_by_month_and_year = create_aggregate_dataframe(us, ['year', 'state', 'month'])
send_dataframe_to_dashboard(world_by_month_and_year, 'aggregate_cym', client)
send_dataframe_to_dashboard(us_by_month_and_year, 'aggregate_sym', client)

The column charts are sightings vs type, with the year selected by the slider and the locale by clicking on the country/state on the map.  

In [9]:
world_by_type_and_year = create_aggregate_dataframe(ufos, ['year', 'country', 'type'])
us_by_type_and_year = create_aggregate_dataframe(us, ['year', 'state', 'type'])
send_dataframe_to_dashboard(world_by_type_and_year, 'aggregate_cyt', client)
send_dataframe_to_dashboard(us_by_type_and_year,  'aggregate_syt', client)